# App objective

The purpose of this app is to poll attendees to an event, e.g. a meeting, and display the results of the polling. Also, we would like to showcase some of the possibilities of using notebooks and voila for creating useful simple apps. 

In this example the attendee is presented with the question: Where would you like to go for vacation?
After typing the desired destination, we send a geocoding request to LocationIQ using the geocoder library to get a list of potential locations.

If we receive a valid response from the request we present the user with the returned list (up to four) of places that match the desired destination on a Select widget. By default, we show on a ipyleaflet Map a Marker located on the coordinates of the first returned location. This Marker position will change based on the user selection from the Selection widget.

Once the user has selected the desired location we store it and then we present the user with the current results of the polling on a map, with one circle (CircleMarker) for each different destination entered, varying the circle radius based on the total number of votes (popularity) for that particular destination.


# Dependencies

If running locally, we need to install these libraries the current environment before executing the next cell:

1. [ipywidget](https://ipywidgets.readthedocs.io/en/latest/user_install.html):
    `conda install -c conda-forge ipywidgets`

2. [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation.html):
    `conda install -c conda-forge ipyleaflet`

3. [geocoder](https://geocoder.readthedocs.io/api.html#installation):
    `pip install geocoder`

4. [voila](https://github.com/QuantStack/voila):
    `conda install voila -c conda-forge`

5. [psycopg2-binary](http://initd.org/psycopg/docs/install.html):
    `pip install psycopg2-binary`

In [1]:
import os
import psycopg2
from ipyleaflet import Map, CircleMarker, Marker, FullScreenControl, WidgetControl, basemaps, basemap_to_tiles
from ipywidgets import HTML, Button, Text, Select, Layout

import geocoder

import warnings

warnings.filterwarnings("ignore")

# Database setup

We have two databases (db) to work with:
1. Local database for developing the code
2. The remote database from Heroku

The code in this section should work for both. The only caveat is the `DATABASE_URL` variable value. Please use the switch below to set it accordingly (`True` use Heroku's db, `False` use local db).

In [2]:
# Make this True for working with Heroku db, False to work with local db
if True:
    DATABASE_URL = os.environ['DATABASE_URL']
else:
    DATABASE_URL = f"dbname={os.environ['USER']}"

In [3]:
def write_vote_to_db(DATABASE_URL):
    '''
    Write vote to the database.
    
    Notes:
        Grab the vote values from the Selection widget using the current index. Store geolocated:
            place_id: LocationIQ unique identifier.
            address: Address as is from the geolocation.
            long: Longitude.
            lat: Latitude.
            count: Number of times this place has been voted. 
    '''
    with psycopg2.connect(DATABASE_URL, sslmode='require') as conn:
        with conn.cursor() as cur:
            cur.execute('''
                        CREATE TABLE IF NOT EXISTS Cities (
                        place_id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
                        address TEXT,
                        long FLOAT,
                        lat FLOAT,
                        count INTEGER);''')

            # Index on select widget
            city_idx = select.index
            # from global variable
            selected_city = geocoded_cities[city_idx]

            place_id = selected_city.place_id
            address = selected_city.address
            long = selected_city.lng
            lat = selected_city.lat

            cur.execute('SELECT count FROM Cities WHERE place_id = (%s);', (place_id, ))
            row = cur.fetchone()
            if row is None:
                cur.execute('''INSERT INTO Cities (place_id, address, long, lat, count)
                        VALUES (%s, %s, %s, %s, 1);''', (place_id, address, long, lat,))
            else:
                cur.execute('UPDATE Cities SET count = count + 1 WHERE place_id = %s;',
                            (place_id,))
            conn.commit()
            cur.close()

In [4]:
def read_votes_in_db(DATABASE_URL):
    '''
    Read votes in database and sort them by decreasing popularity
    '''
    with psycopg2.connect(DATABASE_URL, sslmode='require') as conn:
        with conn.cursor() as cur:
            sqlstr = 'SELECT address, count, lat, long FROM Cities ORDER BY count DESC;'
            cur.execute(sqlstr)
            sorted_cities = cur.fetchall()
            cur.close()
    return sorted_cities

In [5]:
# In case you need to drop the table (debug)
if False:
    cur.execute('DROP TABLE IF EXISTS Cities')

# User interface

## Widgets

First we define the widgets to be used in the map (ipyleaflet):

- **input_city**: A Text field for the user to input the name of a place.

- **geocode_input_city**: A button to process the place typed in the input_city text field. This will trigger a geocoding query. If successful present geocoding results in the select widget, set the select option index to the first element, and then place a marker on the map tied to this option's coordinates; else invite user to input a valid place.

- **select**: A single selection-from-options widget. Selection options are based on the geocoding results. The index will default to the first place returned by the geocoding. If the option is changed, the current marker will be remove, and a new marker will be added to the map.

- **vote**: A button to add the current selected option to the db. After adding to db, display circle markers to show all entries to date, with the circle size proportional to the place's popularity (total number of entries or count). Then, deactivate all widgets to prevent the user from submitting multiple votes, and activate the refresh button, so the user can get and updated view after casting his/her vote.

In [6]:
input_city = Text(
                value="Tahiti",
                placeholder="Type the name of a place",
                description="Where to?",
                disabled=False
            )

geocode_input_city = Button(
                description="Find places",
                disabled=False,
                button_style="success", # 'success', 'info', 'warning', 'danger' or ''
                tooltip="Click me",
                icon='check'
            )

select = Select(
            options=[""],
            value= "",
            description="Select a city:",
            disabled=False,
            )


vote = Button(
              description="Vote!",
              disabled=True,
              button_style="success", # 'success', 'info', 'warning', 'danger' or ''
              tooltip="Click me",
              icon='check'
            )

refresh = Button(
              description="Refresh map",
              disabled=False,
              button_style="success", # 'success', 'info', 'warning', 'danger' or ''
              tooltip="Click me",
              icon='refresh',
              layout=Layout(width='160px', height='80px')
            )

## Map

After successful registration with LocationIQ (more information on the README file) you will be provided with a key. If you are trying to reproduce this app, your key must be stored in the text file (key.txt) inside the notebooks directory, i.e. notebooks/key.txt. **Warning**: Do not to commit your secret key to your github repository if it set to public.

### Coordinate reference system

Both [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html) and LocationIQ use [EPSG 3857](https://epsg.io/3857): WGS 84 / Pseudo-Mercator -- Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI.

### Read the Key

In [7]:
with open("key.txt", "r") as fhandle:
    key = fhandle.readline().split()[0]

### Set buttons to observe changes

In [8]:
def on_geocode_input_city_cliked(button):
    global geocoded_cities
    geocoded_cities = geocoder.locationiq(input_city.value, key=key, maxRows=4)
    
    if geocoded_cities.ok:
        
        select.options = [city.address for city in geocoded_cities]
        
        if geocode_input_city.button_style != "success":
            geocode_input_city.button_style = "success"
        
        select.index = 0
        
        vote.disabled=False
        
    else:
        select.options = ["Please try again"]
        geocode_input_city.button_style = "danger"
        geocode_input_city.description = "Invalid city name"
        
        vote.disabled=True

In [9]:
# revert button style to success, after a bad city input
def on_input_text_selection(change):
    geocode_input_city.button_style = "success"
    geocode_input_city.description = "Find places"
    
    select.options = [""]
    
input_city.observe(on_input_text_selection, names="value")

In [10]:
def remove_map_layer_type(m, layer_type):
    for layer in m.layers:
        if layer.__class__.__name__ == layer_type:
            m.remove_layer(layer)

In [11]:
def on_selection_change(change):
    
    if geocoded_cities.ok:
        remove_map_layer_type(m, "Marker")

        city_idx = select.index
        selected_city = geocoded_cities[city_idx]

        name = selected_city.address
        lat = selected_city.lat
        long = selected_city.lng

        marker = Marker(location=[lat, long])
        m.add_layer(marker)

        message = HTML()
        message.value = name

        marker.popup = message
    
select.observe(on_selection_change, names="index")    

In [12]:
def refresh_plot(m, cities=None):
    # Remove old circles if present
    remove_map_layer_type(m, "CircleMarker")

    sorted_cities = read_votes_in_db(DATABASE_URL)
    
    max_count = sorted_cities[0][1] # First row, second field (count)

    max_radius = 20
    for row in sorted_cities:
        address, count, lat, long = row

        name = address.split(",")[0]

        radius = round((count * max_radius)/ max_count)
        marker = CircleMarker(location=[lat, long], fill_color='black', color='black', radius=radius, weight=2)
        m.add_layer(marker)
        marker.popup = HTML(f"{name}, Popularity: {count}")

In [13]:
def on_vote_clicked(button): 
    # remove selection markers
    remove_map_layer_type(m, "Marker")
    
    write_vote_to_db(DATABASE_URL)
        
    select.disabled = True
    vote.disabled = True
    geocode_input_city.disabled = True
    input_city.disabled = True
    
    refresh_plot(m)
    
    # Remove Widget controls after succesful vote
    for control in m.controls:
        if control.__class__.__name__ == "WidgetControl":
            m.remove_control(control)
    
    # Add refresh button
    widget_control_refresh = WidgetControl(widget=refresh, position="topleft")
    m.add_control(widget_control_refresh)

In [14]:
def on_refresh_clicked(button):
    refresh_plot(m)

In [15]:
m = Map(zoom=2, world_copy_jump=True)

opentopo_layer = basemap_to_tiles(basemaps.OpenTopoMap)
m.add_layer(opentopo_layer)

m.add_control(FullScreenControl())

widget_control_inputText = WidgetControl(widget=input_city, position="topleft")
widget_control_geocode_input_city = WidgetControl(widget=geocode_input_city, position="topleft")
widget_control_select = WidgetControl(widget=select, position="topleft")
widget_control_vote = WidgetControl(widget=vote, position="topleft")

m.add_control(widget_control_inputText)
m.add_control(widget_control_geocode_input_city)
m.add_control(widget_control_select)
m.add_control(widget_control_vote)

geocode_input_city.on_click(on_geocode_input_city_cliked)

vote.on_click(on_vote_clicked)

refresh.on_click(on_refresh_clicked)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …